In [8]:
import glob
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import os
from build import build_model

In [9]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
!bash report.sh
organisms = set(pandas.read_csv("biomass_constrained.txt",index_col=0,header=None).index.to_list())
try:
    done = {i.split(".csv")[0] for i in os.listdir("./cases/overflow/high_prot/")}
except:
    done = set()
zinc_in_polymerase = set(pandas.read_csv("zinc_in_polymerase.txt",index_col=0,header=None).index.to_list())

In [11]:
run_for = ["Bartonella_quintana_Toulouse","Ochrobactrum_anthropi_ATCC_49188"]
len(run_for)

2

In [1]:
# protein
aas = "EX_ala_L(e),EX_ser_L(e),EX_cys_L(e),EX_arg_L(e),EX_ile_L(e),EX_leu_L(e),EX_lys_L(e),EX_his_L(e),EX_asn_L(e),EX_asp_L(e),EX_thr_L(e),EX_glu_L(e),EX_met_L(e),EX_gln_L(e),EX_pro_L(e),EX_val_L(e),EX_phe_L(e),EX_tyr_L(e),EX_gly(e),EX_trp_L(e)"
exchanges = aas.split(",")

In [13]:
from diets import load_model,constrain_exchanges,get_bounds
from overflow import get_overflow
import coralme

conditions = ["high"]
metabolite = "prot"

for c in conditions:
    name = "{}_{}".format(c,metabolite)
    fluxdir = "./cases/fluxes/{}".format(name)
    overflowdir = "./cases/overflow/{}".format(name)
    if not os.path.isdir(fluxdir):
        os.makedirs(fluxdir)
        os.makedirs(overflowdir)

def run(org):
    model = coralme.io.pickle.load_pickle_me_model("./me-models/{}/MEModel-BIO-{}-ME-TS.pkl".format(org,org))
    sol0 = model.solution
    df0 = sol0.to_frame()
    # Exchanges
    rlist = exchanges
    # Run for all conditions
    for condition in conditions:
        name = "{}_{}".format(condition,metabolite)
        dct = {k:-2 for k in rlist if k in model.reactions}
        constrain_exchanges(model,dct)
        model.optimize()
        model.solution.to_frame().to_csv("cases/fluxes/{}/{}.csv".format(name,org))
        get_overflow(model).to_csv("./cases/overflow/{}/{}.csv".format(name,org))
        

In [14]:
NP = min([10,len(run_for)])
pool = mp.Pool(NP,maxtasksperchild=1)
pbar = tqdm(total=len(run_for),position=0,leave=True)
pbar.set_description('Building ({} threads)'.format(NP))
def collect_result(result):
    pbar.update(1)

for org in run_for:
    args = ([org])
    pool.apply_async(run,args, callback=collect_result)
pool.close()
pool.join()

Building (2 threads):   0%|                               | 0/2 [00:00<?, ?it/s]

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmp28f7yk1m.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmpqzzx47ug.lp
Reading time = 0.00 seconds
: 832 rows, 1788 columns, 7876 nonzeros
Read LP format model from file /tmp/tmp4wta3e6o.lp
Reading time = 0.00 seconds
: 854 rows, 1786 columns, 7706 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmp7zzrnf_m.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Iteration	 Solution to check	Solver Status
---------	------------------	-------------
Read LP format model from file /tmp/tmpr_v3rq7h.lp
Reading time = 0.00 seconds
: 1301 rows, 3132 columns, 13778 nonzeros
        1	1.4050280687025918	Not feasible
Read LP format model from file /tmp/tmp6048uv_1.lp
Reading time = 0.01 seconds
: 1316 rows, 3130 columns, 136

Building (2 threads):  50%|███████████           | 1/2 [02:04<02:04, 124.37s/it]

        2	0.7025140343512959	Not feasible
        3	0.3512570171756479	Not feasible
        4	0.1756285085878240	Not feasible
        5	0.0878142542939120	Not feasible
        6	0.0439071271469560	Not feasible
        7	0.0219535635734780	Not feasible
        8	0.0109767817867390	Not feasible
        9	0.0054883908933695	Optimal
       10	0.0082325863400542	Optimal
       11	0.0096046840633966	Optimal
       12	0.0102907329250678	Optimal
       13	0.0106337573559034	Not feasible
       14	0.0104622451404856	Not feasible
       15	0.0103764890327767	Not feasible
       16	0.0103336109789223	Not feasible
       17	0.0103121719519950	Not feasible
       18	0.0103014524385314	Not feasible
       19	0.0102960926817996	Not feasible
       20	0.0102934128034337	Not feasible
       21	0.0102920728642508	Not feasible
       22	0.0102914028946593	Not feasible


Building (2 threads): 100%|██████████████████████| 2/2 [06:43<00:00, 215.38s/it]